In [48]:

# SetRounding gives us setrounding function
using ColorBitstring, SetRounding

In [11]:
x = Float16(2^(-11))

Float16(0.0004883)

In [12]:
printbits(x)

0001000000000000

In [14]:
# 2^(-11) = 2.0^(4-15) * (1.0000000000)_2

0.00048828125

In [15]:
x + x

Float16(0.000977)

In [17]:
printbits(Float16(2^(-10)))

0001010000000000

In [23]:
binarystring(x)

"2^-11 * (1.0000000000)_2"

In [24]:
binarystring(2x)

"2^-10 * (1.0000000000)_2"

In [27]:
binarystring(1+(x+x))

"2^0 * (1.0000000001)_2"

In [29]:
(1+x)+x

Float16(1.0)

In [32]:
binarystring((Float16(1)/3)^2)

"2^-4 * (1.1100011100)_2"

In [36]:
# non-assoc

(1.1 + 1.2) + 1.3, 1.1 + (1.2+1.3)

(3.5999999999999996, 3.6)

In [38]:
binarystring(π)

"2^1 * (1.100100100001111110110101010001000100001011010001100001000110100110001001100011001100010100010111000000011011100000111001101000100101001000000100100111000001000100010100110011111001100011101000000001000001011101111101010011000111011000100111001101100100010…)_2"

In [41]:
Float16(π, RoundUp) # equivalent to fl^up(π)

Float16(3.143)

In [43]:
Float16(π, RoundDown) # equivalent to fl^down(π)

Float16(3.14)

In [46]:
Float16(ℯ, RoundUp)

Float16(2.719)

In [47]:
# by default we round to the nearest

x = Float32(1)
x/3 # round to nearest

0.33333334f0

In [50]:
setrounding(Float32, RoundDown) do
    x/3
end,
setrounding(Float32, RoundUp) do
    x/3
end # Note x/3 in exact arithmetic is guaranteed to live between these two numbers

(0.3333333f0, 0.33333334f0)

In [58]:
# user Taylor series exp(x) ≈ Σ_{k=0}^n x^k/k!
function exp_taylor(x, n)
    ret = 0
    for k = 0:n
        ret = ret + x^k/factorial(big(k))
    end
    return ret
end

exp_taylor (generic function with 1 method)

In [61]:
@time exp_taylor(1.0, 1000) # worked but slow since using variable prec inte

  0.002493 seconds (14.74 k allocations: 962.336 KiB)


2.718281828459045235360287471352662497757247093699959574966967627724076630353416

In [62]:
# user Taylor series exp(x) ≈ Σ_{k=0}^n x^k/k!
function exp_taylor_fast(x, n)
    ret = 0
    summand = 1
    for k = 0:n
        ret += summand
        summand *= x/(k+1) # avoids variable integ
    end
    return ret
end

exp_taylor_fast (generic function with 1 method)

In [65]:
@time exp_taylor_fast(1.0, 100_000_000) # worked but slow since using variable prec inte

  0.294217 seconds


2.7182818284590455

In [66]:
# an issue:
exp_taylor_fast(1.0, 1000) - ℯ # pretty accurate!

4.440892098500626e-16

In [70]:
exp_taylor_fast(-50.0, 100000) - exp(-50) # Why did this FAIL?

2041.8329628976246

In [71]:
# We need to be able to analysis floating point error in order to use algorithms.!

In [73]:
2^-11

0.00048828125

In [75]:
x = -1

0.00048828125